In [1]:
import os
os.chdir('C:\\Users\\ga63key\\Desktop\\introductory_examples')

In [2]:
import gym
from envs.discrete_bs import BSEnv, decode_action, encode_action, encode_wealth, transform_Q_interval_to_Q_numeric, transform_Q_numeric_to_Q_interval, state_to_numeric
from envs import plotting
from utils.discrete_bs.functions import structure_preserving_update

import numpy as np
import pandas as pd
import math
from collections import defaultdict
import itertools
import time
import matplotlib.pyplot as plt
from scipy import optimize

import dill
import pickle

In [3]:
%matplotlib widget

**Action space**  
Actions denote the fraction of wealth invested in the **risky asset**. Actions are discretized with a step size of 10%, i.e.

$$
\mathcal{A}=[0, 0.1, 0.2, \dots, 0.9, 1].
$$

In [4]:
# Define the vector of actions, discrete investment decisions in 10% steps
actions = np.arange(0, 1.01, step=0.1)                  
print("Actions (Investment in risky asset):", actions)  

Actions (Investment in risky asset): [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]


**Wealth discretisation**

In [5]:
lower = 60      # upper limit of the lowest bucket (0, 90]
upper = 160     # lower limit of the highest bucket (upper, +inf)
delta_bin = 5  # bin-size inbetween 
wealth_bins = np.array([0] + np.arange(lower, upper+1, delta_bin).tolist() + [float('Inf')])  # +1 as upper limit is not included
#wealth_bins = np.arange(lower, upper+1, delta_bin)

print(pd.cut(x=[10], bins=wealth_bins, right=False, retbins=True)[0].categories)

IntervalIndex([[0.0, 60.0), [60.0, 65.0), [65.0, 70.0), [70.0, 75.0), [75.0, 80.0) ... [140.0, 145.0), [145.0, 150.0), [150.0, 155.0), [155.0, 160.0), [160.0, inf)],
              closed='left',
              dtype='interval[float64]')


**Discrete-time and discrete wealth Black-Scholes environment**

In [6]:
# Model parameters
'''
:params mu (float):         expected risky asset return
:params sigma (float):      risky asset standard deviation
:params r (float):          risk-less rate of return
:params T (float):          investment horizon
:params dt (float):         time-step size
:params V_0 (float, tuple): initial wealth, if tuple (v_d, v_u) draws initial wealth V(0) uniformly from [v_d, v_u]
:params actions (np.array): possible investment fractions into risky asset
:params wealth_bins (np.array): contains the limits of each wealth bin in ascending order
:params U_2 (callable):     utility function for terminal wealth (default log-utility)
'''

mu=0.06
sigma=0.2
r=0.04
T=5
dt=1
V_0=(50, 150)
batch_size=1

env = BSEnv(mu=mu, sigma=sigma, r=r, T=T, dt=dt, V_0=V_0, actions=actions, wealth_bins=wealth_bins, batch_size=batch_size)

In [7]:
# Simulation of the wealth evolution in the BS market
done = False
print('Observation: {}, Exact_wealth: {}'.format(env.reset(), env.V_t))      # Reset the environment to state (0, 100)

while not done:
    print('Action: \n', decode_action(10, actions))
    next_obs, reward, done, info = env.step(10)              # Take the action 10 (i.e. 100% investment in risky asset) and observe the next state and reward
    print('Observation: {}, Reward: {}, Done: {}, Exact_wealth: {}'.format(next_obs, reward, done, env.V_t)),      

Observation: [[0 Interval(60.0, 65.0, closed='left')]], Exact_wealth: [60.87914066]
Action: 
 1.0
Observation: [[1 Interval(0.0, 60.0, closed='left')]], Reward: [0.], Done: False, Exact_wealth: 55.15174845310204
Action: 
 1.0
Observation: [[2 Interval(65.0, 70.0, closed='left')]], Reward: [0.], Done: False, Exact_wealth: 67.87505143068125
Action: 
 1.0
Observation: [[3 Interval(85.0, 90.0, closed='left')]], Reward: [0.], Done: False, Exact_wealth: 88.44779152145382
Action: 
 1.0
Observation: [[4 Interval(90.0, 95.0, closed='left')]], Reward: [0.], Done: False, Exact_wealth: 94.19409276598667
Action: 
 1.0
Observation: [[5 Interval(125.0, 130.0, closed='left')]], Reward: [4.85112077], Done: True, Exact_wealth: 127.88363811345633


**Epsilon-Greedy Policy**\
Source: https://www.geeksforgeeks.org/q-learning-in-python/#:~:text=Q%2DLearning%20is%20a%20basic,defined%20for%20states%20and%20actions.

In [8]:
def createEpsilonGreedyPolicy(Q, epsilon, num_actions): 
    """ 
    Creates an epsilon-greedy policy based 
    on a given Q-function and epsilon. 
       
    Returns a function that takes the state 
    as an input and returns the probabilities 
    for each action in the form of a numpy array  
    of length of the action space(set of possible actions). 
    """
    def policyFunction(state): 
   
        Action_probabilities = np.ones(num_actions, 
                dtype = float) * epsilon / num_actions 
                  
        best_action = np.argmax(Q[state]) 
        Action_probabilities[best_action] += (1.0 - epsilon) 
        return Action_probabilities 
   
    return policyFunction

In [9]:
# Action-value function parametrisation in a BS market with log-utility function an no consumption.
#def action_func(x, a, b, c):
#    return a * (x**2) + b * x + c

def func(data, a, b, c):
    '''Parametrised action-value function Q in the log-utility case.'''
    
    t   = data[0]
    inv = data[1]
    v   = data[2]
    T   = data[3]
    dt  = data[4]
    
    # See master thesis
    #z = a * (inv**2) + b * inv + np.log(v) + c * (2-t-1)
    z = a * (inv**2) + b * inv + np.log(v) + c * (T-t-dt)
    # Q-values for terminal states are zero
    z[t==T]=0
    
    return  z


def func_partial_derivative_inv(t, v, a, b, c):
    '''Partial derivative w.r.t. a_pi of the parametrised 
       action-value function Q in the log-utility case.
       
    Args:
    - t[float]: trading date
    - v[float]: wealth level
    - a,b,c[float]: fitted parameters
    '''
    
    # optimum criterion: 2*a*inv + b == 0!
    return(-b/(2*a))

**Q-Learning Algorithm**  
The general Q-Learning Algorithm Outline can be found under the link below.  
Source: https://www.geeksforgeeks.org/q-learning-in-python/#:~:text=Q%2DLearning%20is%20a%20basic,defined%20for%20states%20and%20actions.

In [10]:
def qLearning(env, num_episodes, actions, conc_func, conc_learning_rate, concavisation_start, conc_fit_every, initial_params, discount_factor = 1, epsilon = 1): 
    """ 
    Q-Learning algorithm: Off-policy TD control. 
    Finds the optimal greedy policy while improving 
    following an epsilon-greedy policy
    
    Args:
    :params env [gym.environment]: The environement used in the algorithm to sample trajectories.
    :params num_episodes [Int]: Number of training episodes.
    :params actions [np.array]: Numpy array containing the allocation possibilities to the risky asset.
    :params conc_func [Callable]: Function parametrisation used for the update steps in action direction every iteration.
    :params conc_learning_rate [Float]: The step_size used for a step towards the fitted action-values in the update state in action direction.
    :params concavisation_start [Int]: Number of episode, when to start with the updates in action direction as well.
    :params conc_fit_every[Int]: Fits the conc_func every conc_fit_every iteration.
    :params initial_params[list[float]]: Initial parameters for scipy.optimize
    :params discount_factor [Float]: The discount factor.
    :params epsilon [Float]: Random exploration factor in Q-Learning.
    """
       
    # Action value function 
    # A nested dictionary that maps 
    # state -> (action -> action-value). 
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    A = defaultdict(lambda: np.zeros(env.action_space.n))     # Dictionary: Counts visits of state-actions pairs
    P = defaultdict(lambda: [-1, 1, 1])                       # Dictionary: Contains parameters of conc_func for concavisation 
                                                              #             used as initial parameters for the next optimisation
   
    # Keeps track of useful statistics 
    stats = plotting.EpisodeStats( 
        episode_lengths = np.zeros(num_episodes), 
        episode_rewards = np.zeros(num_episodes))     
       
        
    # Create an epsilon greedy policy function 
    # appropriately for environment action space 
    policy = createEpsilonGreedyPolicy(Q, epsilon, env.action_space.n) 
       
        
    # For every episode
    returns=np.array([])
    terminal_wealths = np.array([])
    
    
    for ith_episode in range(num_episodes): 
           
            
        # Reset the environment and pick the first action 
        state = state_to_numeric(env.reset()[0])
        
        
        for t in itertools.count(): 
               
            # get probabilities of all actions from current state 
            action_probabilities = policy(state)
            # choose action according to  
            # the probability distribution 
            action = np.random.choice(np.arange( 
                      len(action_probabilities)), 
                       p = action_probabilities)
            A[state][action] += 1
   
            # take action and get reward, transit to next state 
            # next_states: np.array of length batch_size
            # rewards:     np.array of length batch_size
            # done: True/False
            next_states, rewards, done, _ = env.step(action)
            # next_states: np.array of length batch size, each entry is a tuple (t, mean.v)
            next_states = np.array([state_to_numeric(next_state) for next_state in next_states])
            # mean_reward: float
            mean_reward = np.mean(rewards)
            
   
            # Update statistics 
            stats.episode_rewards[ith_episode] += mean_reward 
            stats.episode_lengths[ith_episode] = t
               
            
            # TD Update 
            # Selects the batch_size next best actions for each next state in next_states
            best_next_actions = np.array([np.argmax(Q[(next_state[0], next_state[1])]) for next_state in next_states])
            # Calculates the Q-values for each of the following states
            next_q_values = np.array([Q[(next_state[0], next_state[1])][best_next_action] for next_state, best_next_action in zip(next_states, best_next_actions)])
            # Calculate the td-target as the mean reward + \beta * mean_next_q_values 
            # (the higher the batch_size the less noise in the target!!!)
            td_target = mean_reward + discount_factor * np.mean(next_q_values)
            # Calculate the delta
            td_delta = td_target - Q[state][action]


            # Update the action-value for the current state
            Q[state][action] += (1/A[state][action]) * td_delta          # Dynamic Learning Rate alpha=1/#visits of state-action pair
                                                                         # ensures convergence see Sutton & Barto eq. (2.7)

                
            
            # Monotone updates
            
            if td_delta > 0:
                states_to_be_updated = [s_tilde for s_tilde in Q.keys() if (s_tilde[0] == state[0]) & (s_tilde[1] > state[1]) & (Q[s_tilde][action] < Q[state][action])]
                # get first state s_hat=(t, V_hat) with V_hat > V^* and Q(s_hat, action) >= Q[state, action]
                s_hat_list = sorted([s for s in Q.keys() if (s[0] == state[0]) & (s[1] > state[1]) & (Q[s][action] >= Q[state][action])], key=lambda x: x[1])
                
                
                # Update states in between via linear interpolation
                if len(s_hat_list) != 0:
                    s_hat = s_hat_list[0]
                    for s_tilde in states_to_be_updated:
                        Q[s_tilde][action] = Q[state][action] + (s_tilde[1] - state[1])*((Q[s_hat][action] - Q[state][action])/(s_hat[1] - state[1]))
               
                
                # No s_hat, hence update all s_tilde equally Q(s_tilde, action) = Q(s^*, action)
                else:
                    states_to_be_updated = [s_tilde for s_tilde in Q.keys() if (s_tilde[0] == state[0]) & (s_tilde[1] > state[1])]
                    for s_tilde in states_to_be_updated:
                        Q[s_tilde][action] = Q[state][action]
            
            
            elif td_delta < 0:
                states_to_be_updated = [s_tilde for s_tilde in Q.keys() if (s_tilde[0] == state[0]) & (s_tilde[1] < state[1]) & (Q[s_tilde][action] > Q[state][action])]
                # get first state s_hat=(t, V_hat) with V_hat < V^* and Q(s_hat, action) <= Q[state, action]
                s_hat_list = sorted([s for s in Q.keys() if (s[0] == state[0]) & (s[1] < state[1]) & (Q[s][action] <= Q[state][action])], key=lambda x: x[1], reverse=True)
                
                
                # Update states in between via linear interpolation
                if len(s_hat_list) != 0:
                    s_hat = s_hat_list[0]
                    for s_tilde in states_to_be_updated:
                        Q[s_tilde][action] = Q[s_hat][action] + (s_tilde[1] - s_hat[1])*((Q[state][action] - Q[s_hat][action])/(state[1] - s_hat[1]))
                
                
                # No s_hat, hence update all s_tilde equally Q(s_tilde, action) = Q(s^*, action)
                else:
                    states_to_be_updated = [s_tilde for s_tilde in Q.keys() if (s_tilde[0] == state[0]) & (s_tilde[1] < state[1])]
                    for s_tilde in states_to_be_updated:
                        Q[s_tilde][action] = Q[state][action]
                        
            else:
                pass
            
            
            # Semi-structure preserving update
            if ith_episode >= concavisation_start:
                if (ith_episode - concavisation_start) % conc_fit_every == 0:
                    Q, initial_params = structure_preserving_update(Q, actions, step_size=conc_learning_rate, func=conc_func, initial_params=initial_params, T=env.T, dt=env.dt)
                        
            
            # done is True if episode terminated    
            if done: 
                returns = np.append(returns, mean_reward)
                terminal_wealths=np.append(terminal_wealths, env.V_t)
                break
                   
            # Continue with the first next_state (simplifies, no need to average over wealths)        
            state = (next_states[0][0], next_states[0][1])
        
        if ith_episode % 10000 == 0:
            print("Episode: {}, Mean Return: {}, Mean Wealth (V_T): {}, Epsilon: {}".format(ith_episode, round(returns.mean(), 3), round(terminal_wealths.mean(), 3), epsilon))
            #print("td_delta:", td_delta)
            returns = np.array([])
            terminal_wealths=np.array([])
            
        # Epsilon-Decay    
        #if (ith_episode % 10000 == 0) & (ith_episode != 0):
        #    epsilon *= 0.98
        #    policy = createEpsilonGreedyPolicy(Q, epsilon, env.action_space.n)
        #    alpha = 0.1
        
        # Alpha-Decay
        #if (ith_episode % 30000 == 0) & (ith_episode != 0):
        #    if alpha > 0.0011:
        #        alpha *= 1/10
            
       
    return Q, stats, A, initial_params

In [11]:
# Training of the Agent
num_episodes = 200000   # Training for 10 mio. Episodes

# Starts timer
t_0 = time.time()

# Starts training
Q, stats, A, fittedParameters = qLearning(env=env, 
                                          num_episodes=num_episodes,
                                          actions=actions,
                                          conc_func=func, 
                                          conc_learning_rate=5e-2, 
                                          concavisation_start=1e+4,
                                          conc_fit_every=1e+3,
                                          initial_params=[-1,0,0],
                                          discount_factor = 1,
                                          epsilon = 0.3)

# Ends timer
t_1 = time.time()

Episode: 0, Mean Return: 4.986, Mean Wealth (V_T): 146.3, Epsilon: 0.3
Episode: 10000, Mean Return: 4.773, Mean Wealth (V_T): 128.229, Epsilon: 0.3
Episode: 20000, Mean Return: 4.773, Mean Wealth (V_T): 128.015, Epsilon: 0.3
Episode: 30000, Mean Return: 4.77, Mean Wealth (V_T): 127.822, Epsilon: 0.3
Episode: 40000, Mean Return: 4.783, Mean Wealth (V_T): 129.278, Epsilon: 0.3
Episode: 50000, Mean Return: 4.779, Mean Wealth (V_T): 128.642, Epsilon: 0.3
Episode: 60000, Mean Return: 4.778, Mean Wealth (V_T): 128.558, Epsilon: 0.3
Episode: 70000, Mean Return: 4.779, Mean Wealth (V_T): 128.715, Epsilon: 0.3
Episode: 80000, Mean Return: 4.776, Mean Wealth (V_T): 128.218, Epsilon: 0.3
Episode: 90000, Mean Return: 4.774, Mean Wealth (V_T): 127.939, Epsilon: 0.3
Episode: 100000, Mean Return: 4.773, Mean Wealth (V_T): 128.2, Epsilon: 0.3
Episode: 110000, Mean Return: 4.777, Mean Wealth (V_T): 128.451, Epsilon: 0.3
Episode: 120000, Mean Return: 4.77, Mean Wealth (V_T): 127.267, Epsilon: 0.3
Episod

In [13]:
# Print running time
print("Process time: {} s".format(t_1 - t_0))

Process time: 1163.2055032253265 s


In [14]:
func_partial_derivative_inv(1, 100, *fittedParameters)

0.5368083663483803

In [15]:
# Transforms the Q-table to interval representation
Q_interval = transform_Q_numeric_to_Q_interval(Q, wealth_bins)


# Prints the learned Action-values for each state + the best action for each state
for key in Q_interval.keys():
    print("Key:", key)
    print("State-Action Values:", Q_interval[key], sep="\n")
    print("Best Action (Investment in risky asset):", decode_action(np.argmax(Q_interval[key]), actions))

Key: (0.0, Interval(100.0, 105.0, closed='left'))
State-Action Values:
[4.90679303 4.91578846 4.92198542 4.92723053 4.93144977 4.93551903
 4.92982142 4.92884779 4.92598142 4.92246039 4.90690925]
Best Action (Investment in risky asset): 0.5
Key: (1.0, Interval(105.0, 110.0, closed='left'))
State-Action Values:
[4.88491374 4.89432019 4.9008117  4.90651586 4.91362937 4.91303656
 4.91552716 4.90747482 4.90435288 4.88707638 4.90013613]
Best Action (Investment in risky asset): 0.6000000000000001
Key: (2.0, Interval(110.0, 115.0, closed='left'))
State-Action Values:
[4.86130606 4.87075222 4.87766287 4.88294402 4.8892674  4.88791401
 4.88838041 4.89076201 4.88252762 4.88017591 4.8678396 ]
Best Action (Investment in risky asset): 0.7000000000000001
Key: (3.0, Interval(115.0, 120.0, closed='left'))
State-Action Values:
[4.83668165 4.84674217 4.85281066 4.85827779 4.86162618 4.86247386
 4.86673394 4.86312958 4.85901411 4.84614512 4.84452401]
Best Action (Investment in risky asset): 0.600000000000

In [13]:
def plot_q_values(Q, actions):
    '''Creates a 3d Wireframe plot of the Q-value function for each state-action pair 
       and adds the predicted action (i.e. argmax_a Q(s,a)
    
    Args:
    :params: Q [dict] A dictionary containing the action-values for each state
    :params: actions [np.array] A np.array containing the possible actions
    '''
    
    
    def fun(x, y, Q, actions, t):
        '''Help function used in plot_q_values'''
        # Returns the Q-values for each state-action pair at time step .
        return np.array([Q[(t,wealth)][encode_action(action, actions)] for action, wealth in zip(x,y)])
    
    times = sorted(list(set([t for t,_ in Q.keys()])))
    for time in times:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        x = actions
        y = np.array(sorted([wealth for t, wealth in Q.keys() if t == time]))
        X, Y = np.meshgrid(x, y)
        zs = np.array(fun(np.ravel(X), np.ravel(Y), Q, actions, time))
        Z = zs.reshape(X.shape)

        # Predicted Actions for each state
        states = [key for key in Q.keys() if key[0] == time]
        predicted_actions = [decode_action(np.argmax(Q[state]), actions) for state in states]
        wealths = [wealth for _, wealth in states]
        predicted_Q_values = [Q[state][np.argmax(Q[state])] for state in states]

        ax.plot_wireframe(X, Y, Z, color="black")

        ax.set_xlabel('investment in risky asset')
        ax.set_ylabel('wealth')
        ax.set_zlabel('Q-values')
        ax.scatter(predicted_actions, wealths, predicted_Q_values, zdir="z", c="red", alpha=1, label="Predicted Actions")
        plt.title("Learned Q-value surface (at t={})".format(time))
        ax.legend()

        plt.show()

In [14]:
plot_q_values(Q, actions)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [85]:
plt.close('all')

In [86]:
def plot_learned_vs_optimal_policy(env, Q, actions):
    '''Plots the learned policy derived from the action-value function Q vs. the optimal policy for logarithmic utility.
    
    Args:
    :params env[gym.environment]: The environment used for collecting samples.
    :params Q[dict]: The Q-Table.
    :params actions[np.array]: np.array containing the possible investment choices.
    '''
    
    # Construct a seperate plot for each time 
    for time in range(1, int(env.num_timesteps)):
        # get the observed wealth levels for each time
        wealth_levels = sorted([wealth for t, wealth in Q.keys() if t == time])
        # Derives the investment choice from the action-value function Q for the given state
        predicted_actions = [decode_action(np.argmax(Q[(time, wealth)]), actions) for wealth in wealth_levels]

        
        # Plots the learned policy
        plt.plot(wealth_levels, predicted_actions, label="learned")
        # Plots the optimal policy for logarithmic utility
        plt.plot(wealth_levels, ((env.mu - env.r)/(env.sigma**2))*np.ones(len(wealth_levels)), "-.", label="optimal")
        plt.title("Learned policy vs. optimal policy (at time t={})".format(time*env.dt))
        plt.xlabel("wealth")
        plt.ylabel("risky asset allocation")
        plt.legend()
        
        plt.show()

In [87]:
plot_learned_vs_optimal_policy(env, Q, actions)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
dill.dump_session('sessions/Discrete_BS_Q_Learning_vectorized_1.db')